In [1]:
%load_ext autoreload
%autoreload 2

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

/Users/daniel/Documents/Work/research/general-task-vectors/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [85]:
import sys
sys.path.append('..')
torch.set_grad_enabled(False)

from src.utils.model_utils import load_gpt_model_and_tokenizer, set_seed, get_submodule
set_seed(32)

In [42]:
model, config = load_gpt_model_and_tokenizer('gpt2')
print(model)
print(config)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2AttentionAltered(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
          (query): WrapperModule()
          (key): WrapperModule()
          (value): WrapperModule()
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
{'

Calculate the average of the activation on the dataset to grasp any pattern that is task-dependant and not token-dependant 

Activations have the following shape (num_layers, num_heads, seq_len, d_head), being each OV circuit output.

Still to answer:
- How this is connected to the residual stream?

Trying to use [nnsight](https://github.com/JadenFiotto-Kaufman/nnsight) and explore to know how it works

In [184]:
# some queries k=1 ICL
queries = [
    'Rome: Italy, Amsterdam:',
    'Paris: France, Berlin:',
    'Madrid: Spain, Lisbon:',
    'Tokyo: Japan, Madrid:',
]

# generate is meant for multi-iteration runs
with model.generate(max_new_tokens=3) as generator:
    # invoke works in a generation context, where operations on inputs and outputs are tracked
    with generator.invoke(queries[2]) as invoker:
        
        for num_layer in range(config['n_layer']):
            activations = get_submodule(
                model, 
                config['layer_name'].split('.')
            )[num_layer]

            # taking the attention 'output'
            print(f'type:\t {activations.attn.output}')
            print(f'shape:\t {activations.attn.output.shape}')
            # output shape: 
            #   (
            #       hidden_state: [b, seq, d_model], 
            #       (
            #           QK?: [b, n_heads, seq, d_head], 
            #           OV?: [b, n_heads, seq, d_head],
            #       )
            #   )
            break

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


type:	 <nnsight.intervention.InterventionProxy object at 0x17dfe9cd0>
shape:	 (torch.Size([1, 7, 768]), (torch.Size([1, 12, 7, 64]), torch.Size([1, 12, 7, 64])))


calculate mean activations

In [188]:
prompt = "Rome: Italy, Amsterdam: Netherlands, Lisbon: Portugal, Madrid:"

with model.invoke(prompt) as invoker:
    pass # no changes to make in the forward pass
logits = invoker.output.logits
logits = logits[0,-1,:] # only over the final token
probs = logits.softmax(dim=-1)


for tok, prob in zip(probs.topk(10).indices, probs.topk(10).values):
    print(f"p({model.tokenizer.decode(tok)}) = {prob:.3f}")

p( Spain) = 0.900
p( Portugal) = 0.028
p( Madrid) = 0.024
p( Italy) = 0.014
p( Argentina) = 0.005
p( Mexico) = 0.003
p( France) = 0.002
p( Uruguay) = 0.002
p( Brazil) = 0.002
p( Real) = 0.001
